In [37]:
!unzip data/drugs_dirty.csv.zip

Archive:  ex/drugs_dirty.csv.zip
replace drugs_dirty.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: drugs_dirty.csv         


In [2]:
!pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 9.5 MB/s eta 0:00:00


In [4]:
import re
import numpy as np
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
import pandas as pd
import optuna

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
df = pd.read_csv('/content/drugs_dirty.csv')

In [7]:
df

,Unnamed: 0,name,text
0,0,Abacavir,"Treatment of HIV-1 infection in adults, adoles..."
1,0,Abacavir and Lamivudine Tablets,Screen for the HLA-B*5701 allele prior to init...
2,0,Abacavir Oral Solution,Screen for the HLA-B*5701 allele prior to init...
3,0,Abacavir Sulfate Tablets,Screen for the HLA-B*5701 allele prior to init...
4,0,"Abacavir, Lamivudine and Zidovudine Tablets",Limitations of Use: \nLimited data exist on th...
...,...,...,...
7927,0,Zyrtec-D 12 Hour,\ntemporarily relieves these symptoms due to h...
7928,0,Zytiga,\nMetastatic castration-resistant prostate can...
7929,0,Zyvana,Zyvana is an orally administered prescription ...
7930,0,Zyvox,Nosocomial pneumonia caused by Staphylococcus ...


In [8]:
df = df.drop(columns=['Unnamed: 0'])

In [10]:
text = df['text'].values

In [11]:
def clean_text(string):
  temp_string = re.sub('http[s]?://(?:[a-zA-Z]|[0–9]|[$-_@.&+]|(?:%[0–9a-fA-F][0–9a-fA-F]))+', '', string)
  clean_string = re.sub(r"[^a-zA-Z ]", " ", temp_string)
  clean_string = re.sub(r" +", " ", clean_string).lower()
  return clean_string

In [12]:
ex = text[0]

In [13]:
ex_sent = nltk.sent_tokenize(ex)[10]

In [14]:
ex_sent

'Triple NRTI regimen of abacavir, tenofovir disoproxil fumarate (tenofovir DF), and either lamivudine or emtricitabine not recommended at any time because of high rate of virologic failure.'

In [15]:
clean_text(ex_sent)

'triple nrti regimen of abacavir tenofovir disoproxil fumarate tenofovir df and either lamivudine or emtricitabine not recommended at any time because of high rate of virologic failure '

In [16]:
lemmatizer = WordNetLemmatizer()

In [17]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [19]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [20]:
def cleaning(content):
  sents = []
  for idx in tqdm(range(len(content))):
    if content[idx] != "":
      for each_sent in nltk.sent_tokenize(str(content[idx])):
        if each_sent != "":
          temp_sent = []
          each_sent = clean_text(each_sent.lower())
          for each_word in each_sent.split():
            if each_word not in stop_words and len(each_word) >= 3:
              temp_sent.append(lemmatizer.lemmatize(each_word))
          if len(temp_sent) >= 5:
            sents.append(temp_sent)
  return sents

In [21]:
edited_text = cleaning(text)

100%|██████████| 7932/7932 [02:47<00:00, 47.23it/s]


In [23]:
def metric(matrix: np.ndarray):
  return np.linalg.cond(matrix)

### ***Поиск оптимальных гиперпараметров***

*Попытаемся найти оптимальные гиперпараметры word2vec в зависимости от полученного числа обусловленности матрицы - чем оно меньше, тем матрцица близка к невырожденной - следовательно пространство векторов слов не избыточно (в связи с дополнительной размерностью), а значит возможны лучшие результаты для дальнейших моделей.*

In [ ]:
def objective(trial):

  vector_size = trial.suggest_categorical('vector_size', [32, 64, 128, 200, 256])
  sg = trial.suggest_categorical('sg', [0, 1])
  window = trial.suggest_categorical('window', [2, 3, 5, 7])
  min_count = trial.suggest_categorical('min_count', [3, 5, 7])
  epochs = trial.suggest_categorical('epochs', [5, 10])

  model = Word2Vec(edited_text, vector_size=vector_size, workers=4, window=window,
                 min_count=min_count, sg=sg)
  matrix = model.wv.vectors
  return metric(matrix)


In [ ]:
direction = 'minimize'
name = 'Word2Vec'
study = optuna.create_study(direction=direction,study_name=name)
study.optimize(objective, n_trials=50)

[I 2023-04-29 16:51:24,135] A new study created in memory with name: Word2Vec
[I 2023-04-29 16:57:06,287] Trial 0 finished with value: 14.087974548339844 and parameters: {'vector_size': 256, 'sg': 1, 'window': 2, 'min_count': 7, 'epochs': 10}. Best is trial 0 with value: 14.087974548339844.
[I 2023-04-29 17:01:26,382] Trial 1 finished with value: 11.022778511047363 and parameters: {'vector_size': 128, 'sg': 1, 'window': 2, 'min_count': 3, 'epochs': 10}. Best is trial 1 with value: 11.022778511047363.
[I 2023-04-29 17:10:53,416] Trial 2 finished with value: 13.776572227478027 and parameters: {'vector_size': 256, 'sg': 1, 'window': 5, 'min_count': 7, 'epochs': 10}. Best is trial 1 with value: 11.022778511047363.
[I 2023-04-29 17:14:53,635] Trial 3 finished with value: 6.28141975402832 and parameters: {'vector_size': 32, 'sg': 1, 'window': 3, 'min_count': 3, 'epochs': 5}. Best is trial 3 with value: 6.28141975402832.
[I 2023-04-29 17:21:05,359] Trial 4 finished with value: 8.5863542556762

In [24]:
# лучшие гиперпараметры
model = Word2Vec(edited_text, vector_size=32, workers=4, window=2,
                 min_count=3, sg=0)

In [25]:
model2 = Word2Vec(edited_text, vector_size=200, workers=4, window=5,
                 min_count=5, sg=1)

In [31]:
model3 = Word2Vec(edited_text, vector_size=50, workers=4, window=2,
                 min_count=3, sg=0)

In [26]:
metric(model.wv.vectors)

3.5460174

In [27]:
metric(model2.wv.vectors)

12.637819

In [32]:
metric(model3.wv.vectors)

3.7358868

In [44]:
# желаемый путь
path = "/models/word2vec.model"

In [45]:
model.save(path)

In [46]:
# файлы для просмотра пространства эмбеддингов в tensorboard
with open("tensorsfp.tsv", 'w+') as tensors:
    with open( "metadatafp.tsv", 'w+') as metadata:
          row  = '\t'.join(["word", "count"])
          metadata.write(row + '\n')
          for word, ind in model.wv.key_to_index.items():
            row = word + '\t' + str(ind)
            metadata.write(row + '\n')
            vector_row = '\t'.join(map(str, model.wv[word]))
            tensors.write(vector_row + '\n')

            if ind == 9999:
              break